In [ ]:
!echo $http_proxy

In [ ]:
import comet_ml

comet_ml.login(project_name="Random_search")

In [ ]:
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255


In [ ]:
print(x_train.shape)

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
def build_model_graph(experiment):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))  # Flatten layer to reshape (28,28,1) to (784,)
    model.add(
        Dense(
            experiment.get_parameter("first_layer_units"),
            activation="sigmoid",
            input_shape=(784,),
        )
    )
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"],
    )
    return model


def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        x_train,
        y_train,
        batch_size=experiment.get_parameter("batch_size"),
        epochs=experiment.get_parameter("epochs"),
        validation_data=(x_test, y_test),
    )


def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Score %s", score)



In [ ]:
# The optimization config:
config = {
    "algorithm": "random",  # Change to random search
    "name": "Optimize MNIST Network with Random Search",
    "spec": {"maxCombo": 15, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {
            "type": "integer",
            "min": 8,
            "max": 64,
        },
        "batch_size": {
            "type": "discrete",
            "values": [8, 16, 32],
        },
        "learning_rate": {
            "type": "float",
            "min": 0.0001,
            "max": 0.01,
        },
    },
    "trials": 1,
}
opt = comet_ml.Optimizer(config)

In [ ]:
for experiment in opt.get_experiments():
    # Log parameters, or others:
    experiment.log_parameter("epochs", 10)

    # Build the model:
    model = build_model_graph(experiment)

    # Train it:
    train(experiment, model, x_train, y_train, x_test, y_test)

    # How well did it do?
    evaluate(experiment, model, x_test, y_test)

    # Optionally, end the experiment:
    experiment.end()

In [ ]:
# Configuration for the optimizer
config = {
    "algorithm": "random",  # Random search for hyperparameter optimization
    "name": "Optimize MNIST Network with Random Search",
    "spec": {"maxCombo": 15, "objective": "minimize", "metric": "loss"},  # Optimization details
    "parameters": {  # Hyperparameter definitions
        "first_layer_units": {"type": "integer", "min": 8, "max": 64},  # Number of units in the first layer
        "batch_size": {"type": "discrete", "values": [8, 16, 32]},  # Batch size options
        "learning_rate": {"type": "float", "min": 0.0001, "max": 0.01},  # Learning rate range
    },
    "trials": 1,  # Number of trials to run
}

# Initialize the optimizer with the given configuration
opt = comet_ml.Optimizer(config)

# Iterate over experiments generated by the optimizer
for experiment in opt.get_experiments():
    experiment.log_parameter("epochs", 10)  # Log fixed parameter: number of epochs

    model = build_model_graph(experiment)  # Build the model with the experiment's parameters
    train(experiment, model, x_train, y_train, x_test, y_test)  # Train the model on the dataset
    evaluate(experiment, model, x_test, y_test)  # Evaluate the model's performance

    experiment.end()  # End the experiment (optional but recommended)
